# Загрузка 

In [ ]:
# загружаем библиотеки
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport

# статистика
from scipy import sparse
import scipy.stats as ss

# отображение
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns
from sklearn import preprocessing 

# работа со времинем
import datetime
from datetime import datetime
from datetime import timedelta

# валидация по датам
from sklearn.model_selection import TimeSeriesSplit

# метрика
from sklearn.metrics import mean_absolute_error as mae

# модели
from sklearn.ensemble import GradientBoostingRegressor as GBR
from fbprophet import Prophet

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error 
from sklearn.linear_model import LinearRegression


import missingno as msno
import re
from matplotlib.pyplot import stackplot
from scipy import stats


# Скачиваем данные

In [ ]:
chronom_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HACK/EVRAZ/data_task1/chronom_test.csv')
chronom_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HACK/EVRAZ/data_task1/chronom_train.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_chronom_data(data):

  # дропаем "Unnamed: 0"
  data.drop(['Unnamed: 0'], axis=1, inplace=True)
  
  # переводим время в datetime
  data['VR_NACH'] = pd.to_datetime(data['VR_NACH'])
  data['VR_KON']  = pd.to_datetime(data['VR_KON'])

  # дропаем все type_oper != межпл.прост. & вн.пл.прост.
  data = data[data.TYPE_OPER == 'опер']

  # удаляем старые года
  data['year'] = pd.to_datetime(data.VR_NACH).dt.year
  data = data[data.year >= 2021]
  data = data.drop(['year'], axis=1)

  # создаем таблицу для вывода
  data_base = pd.DataFrame()

  # считаем колличество кислорода
  data_base['O2_sum'] = data.groupby(['NPLV']).O2.sum()

  # считаем колличество процессов
  data_base['NOP_count'] = data.groupby(['NPLV']).NOP.count()

  # считаем общую продолжительность процессов
  data_base['long_proces'] = data.groupby(['NPLV']).VR_KON.max() - data.groupby(['NPLV']).VR_NACH.min()

  # переводим в секунды
  for i in range(len(data_base)):
    data_base['long_proces'].iloc[i] = data_base['long_proces'].iloc[i].total_seconds()

  # считаем колличество кислорода для С и t
  data_base['O2_C'] = data[data.NOP == 'Додувка на C'].groupby(['NPLV']).O2.sum()
  data_base['O2_t'] = data[data.NOP == 'Додувка на t'].groupby(['NPLV']).O2.sum()

  data_base = data_base.reset_index()
  
  return data_base

In [ ]:
DF = get_chronom_data(chronom_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
DF_T = get_chronom_data(chronom_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
DF

,NPLV,O2_sum,NOP_count,long_proces,O2_C,O2_t
0,510008,2909.0,9,2579,NaN,NaN
1,510009,2182.0,9,4004,NaN,NaN
2,510010,2841.0,9,2904,NaN,NaN
3,510011,10.0,10,3291,NaN,NaN
4,510012,3225.0,11,2895,NaN,1094.0
...,...,...,...,...,...,...
2058,512318,1917.0,10,3084,NaN,1917.0
2059,512319,1660.0,10,3843,1660.0,NaN
2060,512320,0.0,9,3305,NaN,NaN
2061,512321,371.0,10,3660,371.0,NaN


In [ ]:
DF_T

,NPLV,O2_sum,NOP_count,long_proces,O2_C,O2_t
0,512324,1452.0,5,2581,NaN,NaN
1,512327,0.0,3,1766,NaN,NaN
2,512328,0.0,3,3368,NaN,NaN
3,512331,0.0,3,4339,NaN,NaN
4,512333,0.0,3,2773,NaN,NaN
...,...,...,...,...,...,...
775,513369,2433.0,5,2170,NaN,NaN
776,513370,2232.0,5,2101,NaN,NaN
777,513371,1904.0,5,2689,NaN,NaN
778,513372,0.0,3,1614,NaN,NaN


In [ ]:
DF_T.to_csv("df_chronom_test.csv", index=False)
DF.to_csv("df_chronom_train.csv", index=False)

Топ вичи:

1. + общее время процессов
2. + кол. кислорода на ломе 
2. + кол. кислорода уникальныйх процессов (додувка C/t)
3. + общее кол. операций 

5. продолжительность каждого процесса

* наличие уникальныйх процессов (додувка C/t)

https://russianhackers.notion.site/a685453e4fde41a098d9ad704d906e21?v=c482eaeb8f3143d58763b4b9008f1fec